In [1]:
import io, os, re, sys, pandas, requests, json, logging

from time import sleep
from io import StringIO
from datetime import datetime, timedelta, timezone
from dateutil import tz
from dotmap import DotMap
from lxml import etree as ET
from bs4 import BeautifulSoup as BS
from tqdm import tqdm

from Baubles.Colours import Colours
from Perdy.parser import printXML
from Perdy.pretty import prettyPrintLn
from Spanners.Squirrel import Squirrel
from STEP.Converter import Converter, logger
from STEP.Helper import Helper
from STEP.XSD import *
from STEP.REST import ProcessTypes, Processes, Endpoints, Imports, Exports, Classifications, Assets, Entities, Products, Attributes, ObjectTypes
from STEP.SOAP import StepSoapClient

from GoldenChild.xpath import *

In [2]:
dts = '%Y-%m-%d %H:%M:%S %z is %Z'
ltz = tz.gettz('AEDT')

In [5]:
squirrel = Squirrel()

#hostname='https://apacpresales-demo1-step.mdm.stibosystems.com'
hostname='http://dave-steplinux'
#hostname='https://apacpresales-poc-twg.mdm.stibosystems.com'
config_id = 'Stibo Importer'


username='DAVE'
password = squirrel.get(f'stibo:{hostname}:{username}')

context = 'Context1'
workspace = 'Main'

step_soap_client = StepSoapClient(
    hostname=hostname,
    username=username,
    password=password,
)

def constructor(clasz):
    instance = clasz()
    instance.hostname=hostname
    instance.username=username
    instance.password=password
    return instance

process_types   = constructor(ProcessTypes)
processes       = constructor(Processes)
endpoints       = constructor(Endpoints)
imports         = constructor(Imports)
exports         = constructor(Exports)
products        = constructor(Products)
classifications = constructor(Classifications)
entities        = constructor(Entities)
assets          = constructor(Assets)
attributes      = constructor(Attributes)
objecttypes     = constructor(ObjectTypes)

In [6]:
colours = Colours(colour=True)

messages = {		
    'waiting': colours.Blue,
    'running': colours.Cyan,
    'failed': colours.Red,
    'aborted': colours.Magenta,
    'succeeded': colours.Green,
    'completedwitherrors': colours.Orange,
}

def push_it(config_id, file, detailed=False):

    colours = Colours(colour=True)

    now = datetime.now(timezone.utc)
    then = now.astimezone(ltz)

    
    print(f'uploading {file} using config {config_id}')
    response = json.loads(imports.importer(config_id, file, process=f'{then}'))
    bgp_id = response['id']

    print(f'waiting for {bgp_id}')
    waiting = True
    failed = False

    while waiting:
        waiting = False

        bgp = processes.get(bgp_id)

        dts = bgp['ended']
        status = bgp['status']

        if status in ['running', 'waiting']:
            waiting = True
        else:
            failed = True
            errors = processes.report(bgp_id)

        sleep(1)

    if detailed:
        for error in errors:
            if error['entryType'] == 'error':
                failed = True
                print(f'{colours.Orange}{error["entryText"]}{colours.Off}\n')
            else:
                pass
                #sys.stdout.write(f'{error["entryText"]}\n')
            
    #prettyPrintLn(bgp)

    sys.stdout.write(f'{bgp_id}: {bgp["ended"]} -> ')
    sys.stdout.write(f'{messages[str(bgp["status"])]}')
    sys.stdout.write(f'{bgp["status"]}')  
    sys.stdout.write(colours.Off)
    sys.stdout.write('\n\n')

In [7]:
push_it(config_id, 'schema.step.xml', detailed=True)

uploading schema.step.xml using config Stibo Importer
waiting for BGP_417702
BGP_417702: 2024-11-13T10:57:36+11:00 -> succeeded



In [11]:
now = datetime.now(timezone.utc)
then = now.astimezone(ltz)

print(f'{then:%Y-%m-%d %H:%M:%S %Z}')

2024-11-11 12:50:30 AEDT
